# Mining Processes From Smart Contracts

In [ ]:
from web3 import Web3
w3 = Web3(Web3.WebsocketProvider("ws://127.0.0.1:8545", websocket_kwargs={'timeout': 60}))
w3.isConnected()

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
import sha3
k = sha3.keccak_256()
k.update(b"Created(bytes)")
pp.pprint(k.hexdigest())
event_1 = k.hexdigest()
print(event_1)


In [ ]:
CONTRACT_ADDRESS_A = "0x54497Da55F80B125a2050373fb7230e5A4eFA077"
CONTRACT_ADDRESS_B = "0x7F79072c37A02e1F9474aa18Be0B4FeAA68cE624"
PARTICIPANTS_CONTAINER = "0xcd23eCBBdBcC03aE1Bcf658Fd3893F2c6f87DeEf"

ROOT_CONTRACT_ADDRESS = w3.toChecksumAddress("0xbfca45b31a0ea9cc5081693d2f7fe5675449a32c") # first deployed contract from blockchain_etl


In [ ]:
contract_a_filter = w3.eth.filter({
    "fromBlock": "0x0",
    "toBlock": "latest",
    "address": ROOT_CONTRACT_ADDRESS
})

contract_a_eventlist = contract_a_filter.get_all_entries()
pp.pprint(contract_a_eventlist)

In [ ]:
### From BACHOFNER ET AL.
### REWRITE BEFORE COMMITTING

from eth_utils import decode_hex, encode_hex
from rlp import encode

def get_all_transactions_for_contract_address(contract_address=CONTRACT_ADDRESS_A, start_block_number=0, end_block_number=40000):
    contract_address_transactions = list()
    pp.pprint(f"Searching for transactions to/from account {contract_address} within blocks {start_block_number} and {end_block_number}")
    
    for block_number in range(start_block_number, end_block_number+1):
        block = w3.eth.getBlock(block_number, full_transactions=True)
        
        transactions = block["transactions"]
        for transaction in transactions:
            if transaction["from"] == contract_address:
                contract_address_transactions.append(transaction)
                continue
          
            if "to" in transaction and transaction["to"] == contract_address:
                contract_address_transactions.append(transaction)
          
    return contract_address_transactions


def get_all_contract_transactions(start_block_number=0, end_block_number=40000):
    contract_address_transactions_receipts = list()
    contract_address_transactions = list()
    pp.pprint(f"Searching for all contract transactions within blocks {start_block_number} and {end_block_number}")
    
    for block_number in range(start_block_number, end_block_number+1):
        block = w3.eth.getBlock(block_number, full_transactions=True)
        
        transactions = block["transactions"]
        for transaction in transactions:
            trans_rec = w3.eth.getTransactionReceipt(transaction["hash"])
            
            if "contractAddress" in trans_rec and trans_rec["contractAddress"] is not None:
                contract_address_transactions_receipts.append(trans_rec)
                contract_address_transactions.append(transaction)
          
    return contract_address_transactions, contract_address_transactions_receipts

def get_contract_transactions_for_contract_address(contract_address=CONTRACT_ADDRESS_A, start_block_number=0, end_block_number=40000):
    contract_address_transactions_receipts = list()
    contract_address_transactions = list()
    pp.pprint(f"Searching for contract transactions to/from account {contract_address} within blocks {start_block_number} and {end_block_number}")
    
    for block_number in range(start_block_number, end_block_number+1):
        block = w3.eth.getBlock(block_number, full_transactions=True)
        
        transactions = block["transactions"]
        for transaction in transactions:
            trans_rec = w3.eth.getTransactionReceipt(transaction["hash"])
            
            if transaction["to"] == contract_address:
                contract_address_transactions.append(transaction)
                contract_address_transactions_receipts.append(trans_rec)
            if "contractAddress" in trans_rec and trans_rec["contractAddress"] == contract_address:
                contract_address_transactions_receipts.append(trans_rec)
                contract_address_transactions.append(transaction)
          
    return contract_address_transactions, contract_address_transactions_receipts




number = w3.eth.getBlock('latest').number


In [ ]:
# get transaction receipts where contract was involved
trans, recs = get_contract_transactions_for_contract_address(CONTRACT_ADDRESS_A, 0, number)
print("\n Transactions: \n")
pp.pprint(trans) 
print("\n Receipts: \n")
pp.pprint(recs)

In [ ]:
# get all transactions and recipts of all contract transactions
all_trans, all_recs = get_all_contract_transactions(0, number)
print("\n Transactions: \n")
pp.pprint(all_trans)
print("\n Receipts: \n")
pp.pprint(all_recs)

In [ ]:
def normalise(x, allow_blank=False):
    if allow_blank and x == '':
        return ''
    if len(x) in (42, 50) and x[:2] == '0x':
        x = x[2:]
    if len(x) in (40, 48):
        x = decode_hex(x)
    if len(x) == 24:
        assert len(x) == 24 and ssha3.keccak_256(x[:20])[:4] == x[-4:]
        x = x[:20]
    if len(x) != 20:
        raise Exception("Invalid address format: %r" % x)
    return x

def mk_contract_address(sender, nonce):
    normie_sender = normalise(sender)
    res = sha3.keccak_256(encode([normie_sender, nonce])).hexdigest()
    return res[12:]

# from or contract address of previous contract transactions
#contract_address = "0x" + mk_contract_address("0xBF8e90753D5a55ffC3683AB3bf8b7C82Ce52113d", 14)
contract_address = "0x" + mk_contract_address(CONTRACT_ADDRESS_A, 5)
get_all_transactions_for_contract_address(contract_address, 0, number)

In [ ]:
# get transaction receipts where contract was involved
trans, recs = get_contract_transactions_for_contract_address("0xE1EfC7F8F50e2607Ae7c03E27c283339A95373f0", 0, number)
print("\n Transactions: \n")
pp.pprint(trans) 
print("\n Receipts: \n")
pp.pprint(recs)